# Data exploration

In [ ]:
#load library
# load library
import  sys,os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# load data
train_data = read_sql(train_sql)
test_data = read_sql(test_sql)
user_info = read_sql(user_info_sql).drop_duplicates()
user_log = read_sql(user_los_sql).rename(columns = {"seller_id" : 'merchant_id'})


# 查看數據

## 查看數據形態和數據大小

In [ ]:
train_data.head()
test_data.head()
user_info.head()
user_log.head()

In [ ]:
train_data.info()
user_info.info()
user_log.info()


## 查看缺失值

### user_info

In [ ]:
user_info.shape[0]
user_info[user_info['age_range'].isna() | user_info['gender'].isna()].count()

### user_log


In [ ]:
user_log.isna().sum()

# 觀察數據分佈

## user_info describe

In [ ]:
user_info.describe()

## user_log describe

In [ ]:
user_log.describe()

## train label

In [ ]:
label_gp = train_data.groupby('label')['user_id'].count()
print("正負樣本的數量:\n",label_gp )

# 圓餅圖和長條圖
_,axe = plt.subplots(1,2,figsize=(12,6))
train_data.label.value_counts().plot(kind='pie',autopct='%1.1f%%',shadow=True,explode=[0,0.1],ax=axe[0])
sns.countplot('label',data=train_data,ax=axe[1],)

#將訓練集資料正負樣本圓餅圖
train_data.label.value_counts().plot(kind='pie',autopct='%1.1f%%',shadow=True,explode=[0,0.1],ax=axe[0])

#訓練集資料正負樣本長條圖
sns.countplot('label',data=train_data,ax=axe[1],)


# 探索影響購買的各種因素

## 不同店家因素的影響

In [ ]:
train_data_merchant = train_data.copy()
train_data_merchant['TOP5'] = train_data_merchant['merchant_id'].map(
    lambda x: 1 if x in [4044,3828,4173,1102,4976] else 0)

train_data_merchant = train_data_merchant[train_data_merchant['TOP5']==1]

plt.figure(figsize=(8,6))
plt.title('Merchant VS Label')
ax = sns.countplot('merchant_id',hue='label',data=train_data_merchant)

for p in ax.patches:
    height = p.get_height()




In [ ]:
group_by_merchant_label = train_data.groupby(['merchant_id'])['label'].mean()

merchant_repeat_buy = [ rate for rate in train_data.groupby(['merchant_id'])['label'].mean() if rate <= 1 and rate > 0] 

plt.figure(figsize=(8,4))
ax=plt.subplot(1,2,1)
sns.distplot(merchant_repeat_buy, fit=stats.norm)

ax=plt.subplot(1,2,2)
res = stats.probplot(merchant_repeat_buy, plot=plt)


## 使用者性別的影響

In [ ]:
train_data_user_info = train_data.merge(user_info,on=['user_id'],how='left')

plt.figure(figsize=(8,4))
plt.title('Gender  VS Label')
ax = sns.countplot('gender',hue='label',data=train_data_user_info)
for p in ax.patches:
    height = p.get_height()

In [ ]:
gender_repeat_buy_label = train_data_user_info.groupby(['gender'])['label'].mean()
gender_repeat_buy = [ rate for rate in train_data_user_info.groupby(['gender'])['label'].mean()] 


plt.figure(figsize=(8,4))
ax=plt.subplot(1,2,1)
sns.distplot(repeat_buy, fit=stats.norm)

ax=plt.subplot(1,2,2)
res = stats.probplot(gender_repeat_buy, plot=plt)


## 使用者年紀的影響

In [ ]:
age_repeat_buy_label = train_data_user_info.groupby(['age_range'])['label'].mean()
age_repeat_buy = [ rate for rate in train_data_user_info.groupby(['age_range'])['label'].mean()] 


In [ ]:
plt.figure(figsize=(8,4))
ax=plt.subplot(1,2,1)
sns.distplot(age_repeat_buy, fit=stats.norm)

ax=plt.subplot(1,2,2)
res = stats.probplot(age_repeat_buy, plot=plt)


## 重複購買使用者的分佈狀況

In [ ]:
group_by_user_label = train_data.groupby(['user_id'])['label'].mean()
user_repeat_buy = [ rate for rate in train_data.groupby(['user_id'])['label'].mean() if rate <= 1 and rate > 0] 

plt.figure(figsize=(8,4))
ax=plt.subplot(1,2,1)
sns.distplot(user_repeat_buy, fit=stats.norm)

ax=plt.subplot(1,2,2)
res = stats.probplot(user_repeat_buy, plot=plt)

